In [295]:
# Larch imports
from larch.xafs import (pre_edge, autobk, sort_xafs, xftf, xftr, ff2chi, feffpath, 
                        feffit_transform, feffit_dataset, feffit, feffit_report, estimate_noise)
from larch.fitting import param, guess, param_group
from larch.io import read_ascii
from larch.plot.plotly_xafsplots import (plot_mu, plot_bkg, plot_chik, plot_chir, 
                                         plot_chiq, plot_chifit, multi_plot, 
                                         plotlabels, set_label_weight)
import larch.plot.plotly_xafsplots as lp

# General imports
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import copy
import re  

# Plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Jupyter notebook-specific imports
from ipywidgets import interact, widgets
from IPython.display import display, HTML
from ipywidgets import VBox


%matplotlib inline
# %matplotlib widget


## Functions

### For plot

In [296]:
def plot_traces(*figs):
    fig = go.Figure()
    for fig_i in figs:
        fig.add_traces(fig_i.traces)
    fig.show()
    
    
def quad_plots(figs_E, figs_k=None, figs_R=None, figs_q=None, title=None):
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Energy", "k-space", "R-space", "q-space"), horizontal_spacing=0.1, vertical_spacing=0.1)
    fig.update_layout(height=600, width=1000,title_text=title)

    for fig_i in figs_E.traces:
        fig.add_trace(fig_i, row=1, col=1)

    if figs_k is not None:
        for fig_i in figs_k.traces:
            fig.add_trace(fig_i, row=1, col=2)
    
    if figs_R is not None:
        for fig_i in figs_R.traces:
            fig.add_trace(fig_i, row=2, col=1)
    
    if figs_q is not None:
        for fig_i in figs_q.traces:
            fig.add_trace(fig_i, row=2, col=2)

    fig.show()

### For extract parameter value

In [297]:

def extract_par(output, par_str, error_bar=False):
    if not error_bar:
        par_pattern = rf"{par_str}\s+=\s+([\d.]+)"
    else:
        par_pattern = rf"{par_str}\s+=\s+([\d.]+)\s+\+/-\s+([\d.]+)"
    
    match = re.search(par_pattern, output)
    if match:
        if error_bar:
            par_value = float(match.group(1))
            par_uncertainty = float(match.group(2))
            return par_value, par_uncertainty
        else:
            par_value = float(match.group(1))
            return par_value
    else:
        return None

### Search the smoothest data region 

In [298]:
def find_smoothest_range(data, window_size):
    """
    Finds the smoothest range in an oscillation curve based on variance.

    Parameters:
    - data: 1D array or list representing the oscillation curve.
    - window_size: Size of the window for calculating variance.

    Returns:
    - Start and end indices of the smoothest range.
    """
    window_size = int(window_size)  # Ensure window_size is an integer
    variances = np.zeros(len(data) - window_size + 1)

    for i in range(len(variances)):
        variances[i] = np.var(data[i:i+window_size])

    smoothest_index = np.argmin(variances)
    start_index = smoothest_index
    end_index = smoothest_index + window_size - 1

    return start_index, end_index

### Find maximum index of an array

In [299]:
def max_index(arr):
    max_value = arr[0]  # Assume the first element is the maximum
    max_index = 0       # Index of the maximum element
    
    for i in range(1, len(arr)):
        if arr[i] > max_value:
            max_value = arr[i]
            max_index = i
    
    return max_index

##  Step 1: Load data

### Data file

In [300]:

#data_path = "Cufoil_merge.dat"
#data_path = "Cu2O_merge.dat"
#data_path = "CuO_merge.dat"
data_path = "Pt_0p5_ML.dat"
#data_path = "CoNi_1_1_Fresh_Ni-edge.dat"
#data_path = "CoNi_1_1_Reduced_Ni-edge.dat"
#data_path = "Clean-TiO2.dat"
#data_path = "MT-TiO2.dat"

### FEFF file

In [301]:
path = "/Users/cheny8/.larch/feff/"
#cif = "Cu1_K_Copper_cif13096"
#cif = "Cu1_K_Cuprite_cif9326"
#cif = "Cu1_K_Tenorite_cif18820"
cif = "Pt1_L3_Platinum_cif11157"
#cif = "Co1_K_Cobalt_cif15745"
#cif = "Ti1_K_Rutile_cif5167"
first_shell_feff = path + cif + "/" + "feff0001.dat"

### Find data columns

In [302]:
CuFoil = read_ascii(data_path, labels='col1, col2')
CuFoil.is_frozen = False
CuFoil.energy_ref = 'CuFoil'
CuFoil.datatype = 'xas'
CuFoil.xdat = CuFoil.data[0, : ]
CuFoil.ydat = CuFoil.data[1, : ]/1.0
CuFoil.yerr = 1.0
CuFoil.energy = CuFoil.xdat
CuFoil.mu = CuFoil.ydat
sort_xafs(CuFoil, overwrite=True, fix_repeats=True)
CuFoil.groupname = 'CuFoil'
CuFoil.filename = data_path.split(os.sep)[-1]

## Step 2 Normalization + Background

### Data group

In [303]:
figs_CuFoil = {}
label_suffix = 'CuFoil'

# Normalization
pre_edge(CuFoil, pre1=-100, pre2=-20.00, 
         nvict=0, nnorm=2, norm1=50.00, norm2=1000)

# Background function
autobk(CuFoil, 
       rbkg= 1.0, 
       ek0= CuFoil.e0,
       kmin= 0.000, kmax= 16, kweight= 2.0,
       clamp_lo= 0, clamp_hi= 0)

figs_CuFoil['mu'] = plot_mu(CuFoil, show_norm=False, show_flat=False, 
                            show_e0=True, show_pre=True, show_post=True,
                            label=f'mu_{label_suffix}')
figs_CuFoil['bkg'] = plot_bkg(CuFoil, label=f'bkg_{label_suffix}')


In [304]:
fig1 = plot_mu(CuFoil, show_norm=False, show_flat=False,
               show_e0=True, show_pre=True, show_post=True,
               label='mu',offset=0,title=None,win=1,
               show_deriv=False, with_deriv=False, new=False);
fig1.set_style(width=800, height=350) 
indice = np.where(CuFoil.energy>CuFoil.ek0)[0][0]
fig1.add_plot([CuFoil.energy[indice]], [CuFoil.mu[indice]], label='ek0', color='red',linewidth=0, marker=dict(size=5))
fig1.show()

fig2 = plot_chik(CuFoil, show_window=False, kweight=1, delay_draw=True,new=False,
                 kmax=None, win=1, title=None)
fig2.set_style(width=800, height=350)
fig2.show()


## Step 3 Set transform from k to R, find kmax with low noise

### k_max GUI

In [305]:
def generate_kmax_low_noise(noise_factor, noise_threshold, kmax_start, kmax_end, kmax_step, r_window_size, fig = True):

    # Find the smoothest range using kmax_start
    CuFoil_copy=copy.deepcopy(CuFoil)
    xftf(CuFoil_copy, kmin= 3, kmax= kmax_start, dk= 1.000, kweight= 3.000, window='Hanning', rmax_out=12.000)
    # k weight should be 3 to magnifize the noise part.
    start_index, end_index = find_smoothest_range(CuFoil_copy.chir_re, r_window_size)
    r_start_noise = CuFoil_copy.r[start_index]
    r_end_noise = CuFoil_copy.r[end_index] 

    Int_max_noise=[] # Here Int_max is in k space
    epsilon_r_list=[]
    # Plot chir_re for each kmax
    plt.figure(figsize=(6.4,8))
    # For each kmax, do Fourier transform from k to r
    for number in np.arange(kmax_start, kmax_end, kmax_step):
        CuFoil_copy=copy.deepcopy(CuFoil)
        xftf(CuFoil_copy, kmin= 3, kmax= number, dk= 1.000, kweight= 3.000, window='Hanning', rmax_out=12.000) 
        trans = feffit_transform(kmin=3, kmax=number, kweight=[3], dk=1, window='Hanning', rmin=1.5, rmax=3)
        dset = feffit_dataset(data=CuFoil, pathlist=None, transform=trans)
        pars=None
        feffit(pars, dset)
        epsilon_r = dset.epsilon_r[0]
        epsilon_r_list.append(epsilon_r)

        # Import data to data group with indentifier with kmax values 
        indentifier="chir_re_k"+str(number)
        setattr(CuFoil_copy, indentifier, CuFoil_copy.chir_re)

        # Boolean indexing to filter "r" values within the specified range
        mask = (CuFoil_copy.r >= r_start_noise) & (CuFoil_copy.r <= r_end_noise)
        # Extract "chir_re" values corresponding to the filtered "r" range
        chir_re_for_r_range = CuFoil_copy.chir_re[mask]
        # Maximum of absolute chir_re values
        Abs_Max = np.max(np.abs(chir_re_for_r_range))
        Int_max_noise.append(Abs_Max)
        
        if fig:
            plt.subplot(2, 1, 1)
            plt.plot(CuFoil_copy.r, CuFoil_copy.chir_re, label=indentifier)
            plt.axvspan(r_start_noise, r_end_noise, color='yellow', alpha=0.1)
            plt.xlabel('R (Å)') 
            plt.xlim(0,np.max(CuFoil_copy.r))  
        
            plt.ylabel('Re[$\chi(R)$] ($Å^{-3}$)')
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

            plt.subplot(2, 1, 2)
            plt.plot(CuFoil_copy.r, CuFoil_copy.chir_re, label=indentifier)
            plt.xlim(r_start_noise, r_end_noise) 
            plt.xlabel('R (Å)') 
            plt.ylim(-Abs_Max, Abs_Max)  
            plt.ylabel('Re[$\chi(R)$] ($Å^{-3}$)')
            plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    if fig:
        plt.figure()
        plt.plot(np.arange(kmax_start, kmax_end, kmax_step), Int_max_noise, 'o-')
        plt.xlabel('$k_{max}$ ($Å^{-1}$)')
        plt.ylabel('Abs(Re[$\chi(R)])_{max}$ ($Å^{-3}$)')

    noise = list(zip(np.arange(kmax_start, kmax_end, kmax_step), Int_max_noise, epsilon_r_list))

    # Iterate through the noise data
    for i in range(1, len(noise)):
        current_value = noise[i][1]
        first_value = noise[0][1]
        epsilon_r_value = noise[i][2]
    # Noise evaluation
    # noise_factor is usually set as 2
    # noise_threshold is usually set 0.15 at k-weight=3, which is epsilon_r vaulue in the FEFF dataset. Shelly Kelly found this value after comparing good and bad data.  
        if current_value > noise_factor * first_value and epsilon_r_value > noise_threshold:
            kmax_low_noise = noise[i - 1][0]
            break
        else:
            kmax_low_noise = noise[-1][0]

    display(HTML(f"<strong style='color: red; font-size: 16px;'>The kmax for low noise is: {kmax_low_noise}</strong>"))

    return kmax_low_noise

# Define interactive widgets
noise_factor_widget = widgets.FloatText(value=2, description='Noise Factor:', style={'description_width': 'initial'}, layout={'width': '200px'})
noise_threshold_widget = widgets.FloatText(value=0.15, description='Noise Threshold:', style={'description_width': 'initial'}, layout={'width': '200px'})
kmax_start_widget = widgets.FloatText(value=8, description='kmax Start:', style={'description_width': 'initial'},layout={'width': '200px'})
kmax_end_widget = widgets.FloatText(value=np.max(CuFoil.k), description='kmax End:', style={'description_width': 'initial'},layout={'width': '200px'})
kmax_step_widget = widgets.FloatText(value=0.5, description='kmax Step:', style={'description_width': 'initial'},layout={'width': '200px'})
r_window_size_widget = widgets.FloatText(value=30, description='r window size :',style={'description_width': 'initial'}, layout={'width': '200px'})


# Define the interactive function
def on_submit_interactive(noise_factor, noise_threshold, kmax_start, kmax_end, kmax_step, r_window_size):

    # Check if the entered kmax_end value exceeds the limit
    if kmax_end > np.max(CuFoil.k):
        print(f"Error: kmax End should not exceed the maximum value of {np.max(CuFoil.k)}")
        return
    generate_kmax_low_noise(noise_factor, noise_threshold, kmax_start, kmax_end, kmax_step, r_window_size)

# Create interactive widgets container
interactive_widgets = widgets.interactive(on_submit_interactive,
                                          noise_factor=noise_factor_widget,
                                          noise_threshold=noise_threshold_widget,
                                          kmax_start=kmax_start_widget,
                                          kmax_end=kmax_end_widget,
                                          kmax_step=kmax_step_widget,
                                          r_window_size=r_window_size_widget,
                                         )

# Define a title using HTML
title_html = widgets.HTML("<h2 style='color: black;'>kmax with Low Data Noise</h2>")

# Create a VBox to organize the title and interactive widgets vertically
gui_box = widgets.VBox([title_html, interactive_widgets])

# Display the VBox
display(gui_box)


In [306]:
kmax_low_noise = generate_kmax_low_noise(2, 0.15, 8, np.max(CuFoil.k), 0.5, 30, fig = False)

<Figure size 640x800 with 0 Axes>

## Step 4 Fit the 1st shell to find E0 
Using a Path file from FeFF calculation. Here we give math expressions using the Parameters named in the Parameter group to set the values for the Path Parameters in the EXAFS equation.
We will vary amplitude, e0, sigma2, and delta_R, and a third cumulant C3.

In [307]:
xftf(CuFoil, kmin= 3, kmax= kmax_low_noise, dk= 1.000, kweight= 1.000, window='Hanning', rmax_out=12.000)

path1 = feffpath(first_shell_feff, s02='amp', e0='de0', c3='c3', sigma2='sig2', deltar='delr')

pars = param_group(amp=param(0.99, vary=True),
                   de0=param(0, vary=True),
                   c3=param(0, vary=False),
                   sig2=param(0.010, vary=True),
                   delr=param(-0.01, vary=True))

# find the 1st shell peak position in r space
index = max_index(CuFoil.chir_mag)
r_1st_shell = CuFoil.r[index]

# get r range base on the first chir peak.
trans = feffit_transform(kmin=3, kmax=kmax_low_noise, kweight=[1], dk=1, window='Hanning', rmin= r_1st_shell-0.5, rmax=r_1st_shell+0.5)
dset = feffit_dataset(data=CuFoil, pathlist=[path1], transform=trans)
# do the fit...
result = feffit(pars, dset)
# print out the fit report

output = feffit_report(result, with_paths=True)
print(output)

# find deo in the fit result
de0 = result.params["de0"].value

plot_chifit(result.datasets[0], kweight=1, show_real=True)

=================== FEFFIT RESULTS ====================
[[Statistics]]
   n_function_calls   = 32
   n_variables        = 4
   n_data_points      = 66
   n_independent      = 4.81971863
   chi_square         = 36.6283747
   reduced chi_square = 44.6840820
   r-factor           = 0.00689414
   Akaike info crit   = 17.7749082
   Bayesian info crit = 16.0657705
 
[[Dataset]]
   unique_id          = 'dq2hss6r'
   fit space          = 'r'
   r-range            = 1.709, 2.709
   k-range            = 3.000, 9.000
   k window, dk       = 'Hanning', 1.000
   paths used in fit  = ['/Users/cheny8/.larch/feff/Pt1_L3_Platinum_cif11157/feff0001.dat']
   k-weight           = 1
   epsilon_k          = Array(mean=0.0025583, std=8.3032e-4)
   epsilon_r          = 0.0024685
   n_independent      = 4.820
 
[[Variables]]
   amp            =  0.7033770 +/- 0.1507461   (init=  0.9900000)
   c3             =  0.0000000 (fixed)
   de0            =  6.7669982 +/- 1.6391765   (init=  0.0000000)
   delr          

(<larch.plot.plotly_xafsplots.PlotlyFigure at 0x310b22410>,
 <larch.plot.plotly_xafsplots.PlotlyFigure at 0x312a64430>)

### k_min GUI

In [308]:
def generate_kmin_small_r_factor(kmin_start, kmin_end, kmin_step, fig = True):

    pars_fix = param_group(amp=param(0.99, vary=True),
                   de0=param(0, vary=True),
                   c3=param(0, vary=False),
                   sig2=param(0.010, vary=True),
                   delr=param(-0.01, vary=True))
    
    r_factors=[] 
    # Plot chir_re for each kmin
    plt.figure(figsize=(6.4,8))
    # For each kmin, fit the first shell to find r_factor
    for num in np.arange(kmin_start, kmin_end, kmin_step):
        CuFoil_copy=copy.deepcopy(CuFoil)
        CuFoil_copy.energy=CuFoil_copy.energy+de0
        xftf(CuFoil_copy, kmin= num, kmax=kmax_low_noise, dk= 1.000, kweight= 1.000, window='Hanning', rmax_out=12.000)

        # Import data to data group with indentifier with kmin values 
        indentifier="chir_mag_k"+str(num)
        setattr(CuFoil_copy, indentifier, CuFoil_copy.chir_mag)

        trans = feffit_transform(kmin=num, kmax=kmax_low_noise , kweight=[1], dk=1, window='Hanning', rmin=0, rmax=2.5)
        dset = feffit_dataset(data=CuFoil_copy, pathlist=[path1], transform=trans)
        result = feffit(pars_fix, dset)
        output = feffit_report(result, with_paths=True)
        r_factor = extract_par(output, "r-factor", error_bar=False)
        r_factors.append(r_factor)
        
        if fig:
            plot_chifit(dset, kmin=0, kmax=None, rmax=None, show_mag=False, show_real=True, show_imag=False, new=False, win=1)

    if fig:
        #plt.figure()
        #plot_chifit(dset, title='1st-shell fit',rmax=6)  
        
        plt.figure()
        plt.plot(np.arange(kmin_start, kmin_end, kmin_step), r_factors, 'o-')
        plt.xlabel('$k_{max}$ ($Å^{-1}$)')
        plt.ylabel('R-factor')    
  
    
    r_factor_list = list(zip(np.arange(kmin_start, kmin_end, kmin_step), r_factors))
    # Create a list of tuples containing kmin values and r_factors
    r_factor_list = list(zip(np.arange(kmin_start, kmin_end, kmin_step), r_factors))
    # Find the tuple with the smallest r_factors
    min_r_factor_tuple = min(r_factor_list, key=lambda x: x[1])
    # Extract the kmin value from the tuple
    kmin_small_r_factor = min_r_factor_tuple[0]

    display(HTML(f"<strong style='color: red; font-size: 16px;'>The kmin for small R-factor is: {kmin_small_r_factor}</strong>"))

    return kmin_small_r_factor

# Define interactive widgets
kmin_start_widget = widgets.FloatText(value=2, description='kmin Start:', style={'description_width': 'initial'},layout={'width': '200px'})
kmin_end_widget = widgets.FloatText(value=4.1, description='kmin End:', style={'description_width': 'initial'},layout={'width': '200px'})
kmin_step_widget = widgets.FloatText(value=0.5, description='kmin Step:', style={'description_width': 'initial'},layout={'width': '200px'})

# Define the interactive function
def on_submit_interactive(kmin_start, kmin_end, kmin_step):

    # Check if the entered kmin_start value exceeds the limit
    if kmin_start < np.min(CuFoil.k):
        print(f"Error: kmin Start should not exceed the minimum value of {np.min(CuFoil.k)}")
        return
    generate_kmin_small_r_factor(kmin_start, kmin_end, kmin_step)

# Create interactive widgets container
interactive_widgets = widgets.interactive(on_submit_interactive,
                                          kmin_start=kmin_start_widget,
                                          kmin_end=kmin_end_widget,
                                          kmin_step=kmin_step_widget,
                                         )

# Define a title using HTML
title_html = widgets.HTML("<h2 style='color: black;'>kmin with Small R-factor</h2>")

# Create a VBox to organize the title and interactive widgets vertically
gui_box = widgets.VBox([title_html, interactive_widgets])

# Display the VBox
display(gui_box)


## Step 5 Check background/normalization again

In [309]:
# adjust background


## Step 6 Use the k range

In [310]:
kmax_low_noise = generate_kmax_low_noise(2, 0.15, 8, np.max(CuFoil.k), 0.5, 30, fig = False)
kmin_small_r_factor = generate_kmin_small_r_factor(2, 4.1, 0.5, fig = False)

xftf(CuFoil, kmin= kmin_small_r_factor, kmax= kmax_low_noise, dk= 1.000, kweight= 1.000, window='Hanning', rmax_out=12.000)
xftr(CuFoil, rmin= 1.000, rmax= 6.000, dr= 0.500, window='Hanning')

figs_CuFoil['chik'] = plot_chik(CuFoil, kweight=2, label=f'chik_{label_suffix}')
figs_CuFoil['chir'] = plot_chir(CuFoil, label=f'chir_{label_suffix}', show_window=True)
figs_CuFoil['chiq'] = plot_chiq(CuFoil, label=f'chiq_{label_suffix}', show_chik=True, show_window=True)

quad_plots(figs_E=figs_CuFoil['mu'], 
           figs_k=figs_CuFoil['chik'],
           figs_R=figs_CuFoil['chir'],
           figs_q=figs_CuFoil['chiq'])

<Figure size 640x800 with 0 Axes>

<Figure size 640x800 with 0 Axes>